<a href="https://colab.research.google.com/github/GrigoryevV/HF/blob/main/Saiga_Gemma_training_with_Axolotl_Unsloth_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Датасет

In [ ]:
#@title Зависимости датасета

!pip install datasets zstandard jsonlines wandb huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
#@title Логинимся на wandb, чтобы отслеживать запуски

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#@title Генерируем обучающую выборку

from pathlib import Path

!git clone https://github.com/IlyaGusev/rulm.git

self_instruct_dir = Path('rulm/self_instruct').resolve()

content_dir = Path('.').resolve()
!cd {self_instruct_dir} && python -m src.data_processing.create_short_chat_set \
    {content_dir / 'train.jsonl'} \
    {content_dir / 'val.jsonl'}

assert (content_dir / 'train.jsonl').exists()
assert (content_dir / 'val.jsonl').exists()
!head -n 3 train.jsonl

Cloning into 'rulm'...
remote: Enumerating objects: 3165, done.
remote: Counting objects: 100% (635/635), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 3165 (delta 593), reused 579 (delta 565), pack-reused 2530
Receiving objects: 100% (3165/3165), 1.19 MiB | 4.38 MiB/s, done.
Resolving deltas: 100% (2124/2124), done.
Filtering content: 100% (66/66), 54.46 MiB | 10.06 MiB/s, done.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for lksy/ru_instruct_gpt4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/lksy/ru_instruct_gpt4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 100% 15056/15056 [00:00<00:00, 21431.46 examples/s]
100% 15056/15056 [

#Модель

In [ ]:
#@title Скачиваем базовую модель

from huggingface_hub import snapshot_download;
snapshot_download(repo_id="mhenrichsen/gemma-7b", local_dir="gemma-7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

'/content/gemma-7b'

In [ ]:
#@title Патчим токенизатор
!rm -f gemma-7b/tokenizer.model
!sed -i 's/<start_of_turn>/<|im_start|>/g' gemma-7b/tokenizer.json
!sed -i 's/<end_of_turn>/<|im_end|>/g' gemma-7b/tokenizer.json
!sed -i 's/<start_of_turn>/<|im_start|>/g' gemma-7b/tokenizer_config.json
!sed -i 's/<end_of_turn>/<|im_end|>/g' gemma-7b/tokenizer_config.json
!sed -i 's/<start_of_turn>/<|im_start|>/g' gemma-7b/special_tokens_map.json
!sed -i 's/<end_of_turn>/<|im_end|>/g' gemma-7b/special_tokens_map.json

# Axolotl

In [ ]:
#@title Ставим сам axolotl
!pip install torch==2.1.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!git clone https://github.com/OpenAccess-AI-Collective/axolotl
!cd axolotl && pip3 install packaging && pip3 install -e '.[flash-attn]'

Looking in indexes: https://download.pytorch.org/whl/cu121
fatal: destination path 'axolotl' already exists and is not an empty directory.
Obtaining file:///content/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-install-8ylr3af8/peft_af2cc3b93fdb4b7bbea75cf603b8233e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-install-8ylr3af8/peft_af2cc3b93fdb4b7bbea75cf603b8233e
  Resolved https://github.com/huggingface/peft.git to commit 1b3b7b5b2a931e0b49a39b078c5c065c89b088a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.39.0.dev0-py3-none-any.whl
  Running setup.py develop for axolotl


In [ ]:
#@title Конвертируем в ShareGPT формат
import json

def convert_to_sharegpt(input_file, output_file):
    with open(input_file, encoding="utf-8") as r, open(output_file, "w", encoding="utf-8") as w:
        for line in r:
            record = json.loads(line)
            # {"conversations": [{"from": "...", "value": "..."}]}
            role_mapping = {
                "user": "human",
                "system": "system",
                "bot": "gpt"
            }
            messages = [{"from": role_mapping[m["role"]], "value": m["content"]} for m in record["messages"]]
            w.write(json.dumps({"conversations": messages}, ensure_ascii=False).strip() + "\n")

convert_to_sharegpt("train.jsonl", "train_sharegpt.jsonl")
convert_to_sharegpt("val.jsonl", "val_sharegpt.jsonl")

In [ ]:
!head train_sharegpt.jsonl

{"conversations": [{"from": "system", "value": "You are a virtual assistant that wants to be helpful"}, {"from": "human", "value": "pod install -bash: pod: command not found"}, {"from": "gpt", "value": "Please follow the following steps to install Pod:\n\n1. Open terminal and type:\n```sudo gem install cocoapods\n```\nGem will get installed in Ruby inside the System library. Or try on 10.11 Mac OSX El Capitan, type:\n```sudo gem install -n /usr/local/bin cocoapods\n```\nIf there is an error &quot;activesupport requires Ruby version >= 2.xx&quot;, then install the latest active support first by typing in the terminal.\n```sudo gem install activesupport -v 4.2.6\n```\n2. After installation, there will be a lot of messages, read them and if no error found, it means cocoa pod installation is done. Next, you need to set up the cocoa pod master repo. Type in terminal:\n```pod setup\n```\nAnd wait it will download the master repo. The size is very big (370.0MB in Dec 2016). So it can be a whi

In [ ]:
#@title Пишем конфиг
%%writefile gemma_saiga.yaml

base_model: gemma-7b
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  - path: train_sharegpt.jsonl
    type: sharegpt
    conversation: chatml
  - path: val_sharegpt.jsonl
    type: sharegpt
    conversation: chatml

chat_template: chatml
special_tokens:
  bos_token: "<bos>"
  eos_token: "<|im_end|>"
  unk_token: "<unk>"

val_set_size: 0.05

output_dir: ./axolotl_model

adapter: lora
lora_r: 16
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
  - q_proj
  - v_proj
  - k_proj
  - o_proj
  - down_proj
  - up_proj
  - gate_proj

sequence_len: 4096
sample_packing: false
pad_to_sequence_len: false

wandb_project: gemma_test
wandb_name: axolotl_gemma_7b

gradient_accumulation_steps: 128
micro_batch_size: 1
num_epochs: 3
optimizer: adamw_torch
lr_scheduler: cosine
learning_rate: 0.0001

train_on_inputs: false
group_by_length: false
bf16: auto
tf32: false

gradient_checkpointing: true
logging_steps: 1
flash_attention: true

warmup_ratio: 0.05
evals_per_epoch: 4
eval_max_new_tokens: 128
saves_per_epoch: 2
weight_decay: 0.0

default_system_message: "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

Overwriting gemma_saiga.yaml


In [ ]:
#@title Смотрим на токенизацию
!python -m axolotl.cli.preprocess gemma_saiga.yaml --debug

[2024-02-23 16:39:31,912] [INFO] [datasets.<module>:58] [PID:106833] PyTorch version 2.1.1+cu121 available.
[2024-02-23 16:39:31,913] [INFO] [datasets.<module>:95] [PID:106833] TensorFlow version 2.15.0 available.
[2024-02-23 16:39:31,914] [INFO] [datasets.<module>:108] [PID:106833] JAX version 0.4.23 available.
2024-02-23 16:39:32.957335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 16:39:32.957391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 16:39:32.958908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 16:39:34.200367: W tensorflow/compiler/t

In [ ]:
#@title Обучаем
!accelerate launch -m axolotl.cli.train gemma_saiga.yaml

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-02-23 16:39:52,168] [INFO] [datasets.<module>:58] [PID:107047] PyTorch version 2.1.1+cu121 available.
[2024-02-23 16:39:52,169] [INFO] [datasets.<module>:95] [PID:107047] TensorFlow version 2.15.0 available.
[2024-02-23 16:39:52,170] [INFO] [datasets.<module>:108] [PID:107047] JAX version 0.4.23 available.
2024-02-23 16:39:53.179385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 16:39:53.179434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] U

In [ ]:
#@title Сохраняем модель в GDrive
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!cd axolotl_model &&  rm -f saiga_gemma_9b.tar.gz
!cd axolotl_model && tar -czvf saiga_gemma_9b.tar.gz *.json *.bin *.model
!cp saiga_gemma_9b.tar.gz drive/MyDrive/saiga_gemma_9b.tar.gz

adapter_config.json
added_tokens.json
config.json
special_tokens_map.json
tokenizer_config.json
tokenizer.json
adapter_model.bin
tokenizer.model


In [ ]:
#@title Проверяем на нескольких примерах
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained(
    "gemma-7b",
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    "axolotl_model",
    torch_dtype=torch.bfloat16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("axolotl_model")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import GenerationConfig


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config,
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


generation_config = GenerationConfig(
    temperature=0.2,
    do_sample=True,
    top_p=0.95,
    top_k=30,
    max_length=512,
    eos_token_id=tokenizer.eos_token_id
)


system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
inputs = ["Почему трава зеленая?", "Что такое тупой угол?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч", "Что такое глазунья?"]
for inp in inputs:
    prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{inp}<|im_end|>\n<|im_start|>assistant\n"
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Почему трава зеленая?
Зеленый цвет травы обусловлен наличием в её клетках пигмента хлорофилла, который играет важную роль в процессе фотосинтеза – преобразования солнечного света в энергию, используемую для роста и развития растений. Хлорофилл поглощает свет в определенных диапазонах длин волн, в том числе в зеленой области спектра, и отражает свет в других диапазонах, что и обуславливает зелёный цвет.




Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Что такое тупой угол?
Тупой угол - это угол, который больше прямого, но меньше развёрнутого. Он составляет от 90 до 180 градусов. В геометрии тупые углы часто встречаются в треугольниках, где они образуются между двумя сторонами, которые не являются наименьшей и наибольшей.

Например, если у вас есть треугольник с углами 30 градусов, 40 градусов и 110 градусов, то угол в 110 градусов будет тупым.

Тупые углы также встречаются в других геометрических фигурах, таких как четырехугольники, где они могут образовываться между двумя соседними сторонами.

В everyday life тупые углы можно встретить в различных предметах, таких как стул, где угол между ножками и сиденьем является тупым, или в дверных проёмах, где угол между дверью и стеной также является тупым.

Важно отметить, что тупые углы не являются прямыми, но и не являются острыми. Они находятся между этими двумя типами углов.




Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды, в далекой стране, жила маленькая девочка по имени Таня. Она была очень активной и любила играть в мяч. Вместе со своими друзьями они проводили дни, играя в футбол, баскетбол и волейбол.

Таня была очень талантливой и ловкой, и она часто выигрывала в играх с другими детьми. Но однажды, во время игры в баскетбол, она получила сильную травму. Ей пришлось провести несколько недель в больнице, и она была очень грустной и унылой.

Её друзья не могли понять, как помочь Тане справиться с этой ситуацией. Они пытались поддерживать её, но Таня была слишком подавлена и не хотела играть в мяч.

Однажды, Таня увидела, как её друзья играют в мяч. Она не могла не смотреть, как они ловко передают мяч друг другу и забивают голы. Таня почувствовала, как её сердце разрывается от желания играть вместе с ними.

Она решила, что ей нужно вернуться в игру, чтобы снова почувствовать радость игры. Она начала медленно восстан

In [ ]:
#@title Логинимся на хаб
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title Сливаем с базовой моделью и заливаем на хаб

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = "axolotl_model"
config = PeftConfig.from_pretrained(model_name)
base_model_path = config.base_model_name_or_path

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    load_in_8bit=False,
    device_map="cpu",
    torch_dtype=torch.bfloat16
)

lora_model = PeftModel.from_pretrained(
    base_model,
    model_name,
    device_map="cpu",
    torch_dtype=torch.bfloat16
)

lora_model = lora_model.merge_and_unload()
lora_model.train(False)
lora_model.push_to_hub("IlyaGusev/saiga_gemma_9b")

tokenizer = AutoTokenizer.from_pretrained("axolotl_model")
tokenizer.push_to_hub("IlyaGusev/saiga_gemma_9b")

generation_config = GenerationConfig(
    temperature=0.3,
    do_sample=True,
    top_p=0.95,
    top_k=30,
    max_length=512,
    eos_token_id=tokenizer.eos_token_id
)
generation_config.push_to_hub("IlyaGusev/saiga_gemma_9b")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/IlyaGusev/saiga_gemma_9b/commit/60c62e4bb839a4d51c5ca30a0db3a9b750456c76', commit_message='Upload GemmaForCausalLM', commit_description='', oid='60c62e4bb839a4d51c5ca30a0db3a9b750456c76', pr_url=None, pr_revision=None, pr_num=None)

# Unsloth

In [ ]:
#@title Ставим сам Unlsoth
!pip install torch==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121

import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-gttr_u0c/unsloth_6be135ac9ecc4173af5d668e54e78297
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-gttr_u0c/unsloth_6be135ac9ecc4173af5d668e54e78297
  Resolved https://github.com/unslothai/unsloth.git to commit f946bed7b3b2f1fbee77838f96e59b9a94494790
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
#@title Загружаем модель
from unsloth import FastLanguageModel

model_name = "gemma-7b"
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_8bit=False,
    load_in_4bit=True
)

==((====))==  Unsloth: Fast Gemma patching release 2024.2
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=1337,
    max_seq_length=max_seq_length
)

Unsloth 2024.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
#@title Готовим датасет
#Это необязательно, можно подавать и текстом
%%writefile rulm/self_instruct/internal_prompts/chaml.json

{
    "system_prompt": "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.",
    "system_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "user_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "bot_message_template": "<|im_start|>{role}\n{content}<|im_end|>\n",
    "user_role": "user",
    "bot_role": "assistant",
    "system_role": "system",
    "suffix": "<|im_start|>assistant\n"
}

Overwriting rulm/self_instruct/internal_prompts/chaml.json


In [ ]:
tokenizer.add_special_tokens({'additional_special_tokens': ["<|im_start|>", "<|im_end|>"]})
tokenizer.eos_token = "<|im_end|>"

In [ ]:
%cd rulm/self_instruct

import json

from src.dataset import ChatDataset

mapping = {
    "bot": "assistant"
}
with open("../../train.jsonl") as r:
    train_records = [json.loads(line) for line in r]
    for r in train_records:
        r["messages"] = [{"content": m["content"], "role": mapping.get(m["role"], m["role"])} for m in r["messages"]]

train_dataset = ChatDataset(
    train_records,
    tokenizer,
    max_tokens_count=max_seq_length,
    templates_path="internal_prompts/chaml.json",
    only_target_loss=True
)

with open("../../val.jsonl") as r:
    val_records = [json.loads(line) for line in r]
    for r in val_records:
        r["messages"] = [{"content": m["content"], "role": mapping.get(m["role"], m["role"])} for m in r["messages"]]

val_dataset = ChatDataset(
    val_records,
    tokenizer,
    max_tokens_count=max_seq_length,
    templates_path="internal_prompts/chaml.json",
    only_target_loss=True
)

%cd ../../

/content/rulm/self_instruct


  0%|          | 0/4161 [00:00<?, ?it/s]

[2, 106, 9020, 108, 2045, 708, 476, 10660, 20409, 674, 8507, 577, 614, 10055, 107, 108, 106, 1645, 108, 12286, 4877, 728, 17406, 235292, 3478, 235292, 5017, 780, 1942, 107, 108, 106, 105776, 108, 5958, 1611, 573, 2412, 7161, 577, 4877, 13731, 235292, 109, 235274, 235265, 6376, 17145, 578, 1916, 235292, 108, 1917, 27879, 21737, 4877, 22355, 11649, 2342, 108, 1917, 108, 39761, 877, 947, 11598, 575, 39671, 5580, 573, 1479, 9581, 235265, 2364, 3418, 611, 235248, 235274, 235276, 235265, 235274, 235274, 5439, 163004, 2810, 174812, 235269, 1916, 235292, 108, 1917, 27879, 21737, 4877, 728, 235254, 1148, 13774, 235283, 5047, 235283, 3049, 22355, 11649, 2342, 108, 1917, 108, 2495, 1104, 603, 671, 3333, 724, 14177, 235289, 550, 6997, 46140, 9286, 39671, 3797, 5843, 235248, 235284, 235265, 6224, 235343, 14177, 49298, 1492, 4877, 573, 7776, 6066, 2676, 1370, 731, 28740, 575, 573, 17145, 235265, 108, 1917, 27879, 21737, 4877, 209054, 46140, 728, 235272, 235248, 235310, 235265, 235284, 235265, 235318

 45%|████▍     | 98/220 [00:00<00:00, 488.92it/s]

[2, 106, 9020, 108, 64748, 4590, 1215, 12575, 121729, 235339, 235265, 4590, 1215, 12575, 121729, 235339, 728, 204006, 7654, 55315, 205854, 975, 24628, 54844, 17437, 3264, 235401, 981, 38603, 8295, 41345, 235269, 180244, 1333, 1416, 55522, 136319, 6848, 77976, 235269, 4755, 941, 12626, 90763, 230050, 27301, 14436, 4171, 3628, 5788, 72772, 235265, 4763, 4962, 1391, 1822, 975, 29544, 115866, 7654, 74935, 1153, 115788, 87883, 93031, 3529, 235269, 43958, 198571, 1199, 129782, 3086, 7654, 71530, 31437, 41345, 235265, 34263, 14436, 3229, 11938, 40498, 2186, 137028, 29603, 8445, 235269, 69032, 203631, 3562, 18011, 1153, 33820, 56422, 3025, 198646, 133745, 235375, 235265, 36020, 83176, 941, 141702, 1153, 107033, 8295, 41345, 2202, 235269, 29523, 61657, 74659, 93031, 3529, 1153, 2087, 1416, 163889, 64788, 121266, 235265, 107, 108, 106, 1645, 108, 104934, 235269, 4590, 1215, 12575, 121729, 235339, 235341, 93225, 5353, 3562, 20517, 128225, 6108, 9209, 63033, 1199, 97265, 48006, 2974, 125581, 23526

100%|██████████| 220/220 [00:00<00:00, 519.97it/s]

/content


In [ ]:
#@title Обучаем
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

trainer = Trainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = data_collator,
    args = TrainingArguments(
        num_train_epochs = 3,
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 2,
        gradient_accumulation_steps = 64,
        warmup_steps = 4,
        learning_rate = 0.0002,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        eval_steps = 8,
        save_steps = 8,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 1337,
        output_dir = "unsloth_model",
        evaluation_strategy = "steps",
        load_best_model_at_end = True
    ),
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,161 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 64
\        /    Total batch size = 128 | Total steps = 96
 "-____-"     Number of trainable parameters = 50,003,968
wandb: Currently logged in as: ilyagusev. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
8,1.773500,1.678962
16,1.656000,1.616555
24,1.598400,1.583951
32,1.548600,1.566585
40,1.580300,1.553748
48,1.461600,1.547319
56,1.449200,1.542480
64,1.592800,1.538878
72,1.375200,1.542217
80,1.392000,1.539357


Checkpoint destination directory unsloth_model/checkpoint-8 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory unsloth_model/checkpoint-16 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Step,Training Loss,Validation Loss
8,1.773500,1.678962
16,1.656000,1.616555
24,1.598400,1.583951
32,1.548600,1.566585
40,1.580300,1.553748
48,1.461600,1.547319
56,1.449200,1.542480
64,1.592800,1.538878
72,1.375200,1.542217
80,1.392000,1.539357


TrainOutput(global_step=96, training_loss=1.5347340268393357, metrics={'train_runtime': 3711.4759, 'train_samples_per_second': 3.363, 'train_steps_per_second': 0.026, 'total_flos': 4.579875543331799e+17, 'train_loss': 1.5347340268393357, 'epoch': 2.95})

In [ ]:
#@title Проверяем на нескольких примерах
from transformers import GenerationConfig


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config,
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

FastLanguageModel.for_inference(model)

generation_config = GenerationConfig(
    temperature=0.2,
    do_sample=True,
    top_p=0.95,
    top_k=30,
    max_length=512,
    eos_token_id=tokenizer.eos_token_id
)

system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
inputs = ["Почему трава зеленая?", "Что такое тупой угол?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч", "Что такое глазунья?"]
for inp in inputs:
    prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{inp}<|im_end|>\n<|im_start|>assistant\n"
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Почему трава зеленая?
Цвет травы, а также других растений, обусловлен наличием в их клетках пигмента, называемого хлорофиллом. Хлорофилл поглощает свет в определенных диапазонах длин волн (в основном, в синем и красном диапазонах), и отражает свет в зеленом диапазоне. Таким образом, трава кажется зеленой.

Хлорофилл играет важную роль в процессе фотосинтеза, который позволяет растениям получать энергию из солнечного света. В процессе фотосинтеза хлорофилл поглощает свет и использует его для преобразования углекислого газа и воды в глюкозу (углеводы) и кислород.

Таким образом, зелёный цвет травы обусловлен наличием хлорофилла, который является важным компонентом фотосинтеза и позволяет растениям получать энергию из солнечного света.




Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Что такое тупой угол?
В геометрии, тупой угол - это угол, который больше прямого угла (90 градусов), но меньше полного угла (180 градусов). В других словах, тупой угол - это угол, который больше 90 градусов, но меньше 180 градусов.

Например, если угол составляет 120 градусов, то это тупой угол. Если угол составляет 100 градусов, то это также тупой угол.

Обратите внимание, что угол меньше 90 градусов называется острым углом, а угол, равный 90 градусам, называется прямым углом.




Setting `pad_token_id` to `eos_token_id`:107 for open-end generation.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды, в далеком городе, жила маленькая девочка по имени Таня. Она была очень активной и любила проводить время на улице. Таня часто играла в футбол с мальчишками, и ее любимым предметом был мяч.

Таня была очень дружна и всегда была готова помочь своим друзьям. Однажды, когда она играла в футбол, мяч попал в окно одного из домов на улице. Таня не могла остаться равнодушной и решила помочь своему другу-мальчишке вернуть мяч.

Она подошла к окну и попросила женщину, которая жила там, вернуть мяч. Женщина была очень милой и согласилась помочь. Таня и ее друг-мальчик были очень благодарны ей и решили отблагодарить женщину.

Они вернулись домой и принесли ей подарок - свежий и вкусный пирог. Женщина была очень рада такому подарку и сказала, что очень любит пирог. Она также рассказала Тане, что у нее есть маленькая дочь, которая тоже любит играть в футбол.

Таня была рада узнать о такой девочке и решила, что к